# Raspador de dados do judiciário paulista
Código para raspar dados do E-SAJ do Tribunal de Justiça do Estado de São Paulo (TJ-SP). 

*Matheus Arruda

# Exemplos de uso
### Quais deputados se autobeneficiaram ao votar a favor da lei de improbidade administrativa
O código vai buscar informações, via API da Câmara dos Deputados, da votação do PL n° 2505, de 2021 - sob ID DE VOTAÇÃO: 2184458-184. <br>
Na 2ª etapa, iremos automatizar searchs no e-SAJ do TJ-SP e raspar as informações. 

# Preparando o ambiente
Para a raspagem dos dados, utilizaremos as bibliotecas Selenium. Para a análise, utilizaremos o Pandas. 
Já para a representação dos dados utilizaremos o matplotlib e Seaborn. 

In [13]:

#Biblioteca para raspagem 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

#Bibliotecas para várias funções relacionadas à tempo
import time
from datetime import datetime

#Bibliotecas para análise dos dados
import pandas as pd 
import numpy as np 
import requests as requests
import matplotlib.pyplot as plt 
import seaborn as sns


In [14]:
#Linhas para configurar a IDE para exibir o máximo de colunas e largura máxima de colunas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Conexão API Câmara dos Deputados

In [15]:
#Vamos criar nosso cabeçalho para aceitar as requisiões em json
headers = {
    "Accept": "Application/json"
}

### Identificar os id's da PL

In [16]:
votos_pl = requests.get('https://dadosabertos.camara.leg.br/api/v2/proposicoes/2184458/votacoes?ordem=DESC&ordenarPor=dataHoraRegistro', headers=headers)
votos_pl = votos_pl.json()

votos_pl = pd.json_normalize(votos_pl['dados'])
votos_pl.head(6)


,id,uri,data,dataHoraRegistro,siglaOrgao,uriOrgao,uriEvento,proposicaoObjeto,uriProposicaoObjeto,descricao,aprovacao
0,2184458-186,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-186,2021-10-06,2021-10-06T16:58:25,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63505,None,None,"Aprovada a Redação Final assinada pelo Relator, Dep. Carlos Zarattini (PT-SP).",1.0
1,2184458-184,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-184,2021-10-06,2021-10-06T16:57:25,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63505,None,None,Aprovada a Emenda do Senado Federal nº 8. Sim: 287; não: 133; abstenção: 2; total: 422.,1.0
2,2184458-172,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-172,2021-10-05,2021-10-05T20:05:40,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63472,None,None,"Rejeitada a Emenda do Senado Federal nº 4 ao Projeto de Lei nº 2.505, de 2021, ressalvados os destaques. Sim: 162; não: 253; abstenção: 4; total: 419.",0.0
3,2184458-169,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-169,2021-10-05,2021-10-05T19:47:45,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63472,None,None,"Aprovadas as Emendas do Senado Federal nºs 1, 2, 3, 5, 6, 7 e 8 ao Projeto de Lei nº 2.505, de 2021, ressalvados os destaques. Sim: 395; não: 22; abstenção: 3; total: 420.",1.0
4,2184458-161,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-161,2021-10-05,2021-10-05T18:34:35,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63472,None,None,Rejeitado o Requerimento. Sim: 60; não: 346; total: 406.,0.0
5,2301348-3,https://dadosabertos.camara.leg.br/api/v2/votacoes/2301348-3,2021-10-05,2021-10-05T13:23:52,CPD,https://dadosabertos.camara.leg.br/api/v2/orgaos/537480,https://dadosabertos.camara.leg.br/api/v2/eventos/63437,REQ 76/2021 CPD,https://dadosabertos.camara.leg.br/api/v2/proposicoes/2301348,Aprovado.,1.0


### Lista votos

In [17]:
lista_votos = requests.get('https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-184/votos', headers=headers)
lista_votos = lista_votos.json()

lista_votos = pd.json_normalize(lista_votos['dados'])
lista_votos

,tipoVoto,dataRegistroVoto,deputado_.id,deputado_.uri,deputado_.nome,deputado_.siglaPartido,deputado_.uriPartido,deputado_.siglaUf,deputado_.idLegislatura,deputado_.urlFoto,deputado_.email
0,Sim,2021-10-06T16:55:06,74352,https://dadosabertos.camara.leg.br/api/v2/deputados/74352,Nilson Pinto,PSDB,https://dadosabertos.camara.leg.br/api/v2/partidos/36835,PA,56,https://www.camara.leg.br/internet/deputado/bandep/74352.jpg,None
1,Sim,2021-10-06T16:54:47,74366,https://dadosabertos.camara.leg.br/api/v2/deputados/74366,João Campos,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/partidos/37908,GO,56,https://www.camara.leg.br/internet/deputado/bandep/74366.jpg,None
2,Sim,2021-10-06T16:54:10,204553,https://dadosabertos.camara.leg.br/api/v2/deputados/204553,Pastor Sargento Isidório,AVANTE,https://dadosabertos.camara.leg.br/api/v2/partidos/36898,BA,56,https://www.camara.leg.br/internet/deputado/bandep/204553.jpg,dep.pastorsargentoisidorio@camara.leg.br
3,Não,2021-10-06T16:54:02,141553,https://dadosabertos.camara.leg.br/api/v2/deputados/141553,Vanderlei Macris,PSDB,https://dadosabertos.camara.leg.br/api/v2/partidos/36835,SP,56,https://www.camara.leg.br/internet/deputado/bandep/141553.jpg,None
4,Não,2021-10-06T16:53:57,204363,https://dadosabertos.camara.leg.br/api/v2/deputados/204363,Osires Damaso,PSC,https://dadosabertos.camara.leg.br/api/v2/partidos/36833,TO,56,https://www.camara.leg.br/internet/deputado/bandep/204363.jpg,None
...,...,...,...,...,...,...,...,...,...,...,...
418,Não,2021-10-06T16:45:03,204548,https://dadosabertos.camara.leg.br/api/v2/deputados/204548,Heitor Freire,PSL,https://dadosabertos.camara.leg.br/api/v2/partidos/36837,CE,56,https://www.camara.leg.br/internet/deputado/bandep/204548.jpg,None
419,Sim,2021-10-06T16:45:01,76874,https://dadosabertos.camara.leg.br/api/v2/deputados/76874,Marcelo Freixo,PSB,https://dadosabertos.camara.leg.br/api/v2/partidos/36832,RJ,56,https://www.camara.leg.br/internet/deputado/bandep/76874.jpg,None
420,Sim,2021-10-06T16:45:01,160556,https://dadosabertos.camara.leg.br/api/v2/deputados/160556,Padre João,PT,https://dadosabertos.camara.leg.br/api/v2/partidos/36844,MG,56,https://www.camara.leg.br/internet/deputado/bandep/160556.jpg,dep.padrejoao@camara.leg.br
421,Sim,2021-10-06T16:44:59,73801,https://dadosabertos.camara.leg.br/api/v2/deputados/73801,Renildo Calheiros,PCdoB,https://dadosabertos.camara.leg.br/api/v2/partidos/36779,PE,56,https://www.camara.leg.br/internet/deputado/bandep/73801.jpg,dep.renildocalheiros@camara.leg.br


### Informações adicionais dos deputados

In [ ]:
nome_deputado = []

for i, row in lista_votos.iterrows():
    deputado_id = row['deputado_.id']
    deputados_infos = requests.get(f'https://dadosabertos.camara.leg.br/api/v2/deputados/{deputado_id}')
    deputados_infos = deputados_infos.json()
    nome_deputado.append(deputados_infos['dados']['nomeCivil'])
    print(deputado_id)

lista_votos['NomeCivilDeputado'] = nome_deputado

74352
74366
204553
141553
204363
160758
204505
178934
178884
217480
204547
204414
178896
178927
204364
204387
204531
160592
141516
204473
178996
204482
160591
204395
141513
141458
178986
141434
157130
204559
160976
196358
178937
178956
204527
74090
204398
204428
141459
74317
204377
204515
178933
73692
204539
160610
198783
204392
74376
166402
160553
179001
74537
74253
160527
217330
74439
178910
74646
204571
73486
74052
209189
204449
204431
178964
178921
204488
66179
90201
204386
74159
197438
178970
204534
164359
160601
178985
178912
74158
204511
204360
204408
204475
73466
74419
178873
160528
204502
141405
204467
178843
204518
178871
73788
160510
133439
160508
177282
204532
73604
204427
178909
204522
204446
141391
204410
160604
191945
204479
204405
178835
195866
178948
204369
215042
204413
160666
204481
160642
204366
204537
204485
74749
141523
141408
143084
160665
204407
75431
204497
204538
205548
115746
204566
92776
178939
204418
178832
204393
204498
74467
194260
74254
204372
136811
204

In [ ]:
#Vamos criar um DataFrame contendo apenas os deputados de SP
bd_deputados_sp = lista_votos.query('`deputado_.siglaUf` == "SP" and tipoVoto == "Sim"')
bd_deputados_sp

,tipoVoto,dataRegistroVoto,deputado_.id,deputado_.uri,deputado_.nome,deputado_.siglaPartido,deputado_.uriPartido,deputado_.siglaUf,deputado_.idLegislatura,deputado_.urlFoto,deputado_.email,NomeCivilDeputado
26,Sim,2021-10-06T16:51:53,178986,https://dadosabertos.camara.leg.br/api/v2/deputados/178986,Nilto Tatto,PT,https://dadosabertos.camara.leg.br/api/v2/partidos/36844,SP,56,https://www.camara.leg.br/internet/deputado/bandep/178986.jpg,dep.niltotatto@camara.leg.br,NILTO IGNACIO TATTO
30,Sim,2021-10-06T16:51:37,160976,https://dadosabertos.camara.leg.br/api/v2/deputados/160976,Tiririca,PL,https://dadosabertos.camara.leg.br/api/v2/partidos/37906,SP,56,https://www.camara.leg.br/internet/deputado/bandep/160976.jpg,dep.tiririca@camara.leg.br,FRANCISCO EVERARDO TIRIRICA OLIVEIRA SILVA
34,Sim,2021-10-06T16:51:12,204527,https://dadosabertos.camara.leg.br/api/v2/deputados/204527,Geninho Zuliani,DEM,https://dadosabertos.camara.leg.br/api/v2/partidos/36769,SP,56,https://www.camara.leg.br/internet/deputado/bandep/204527.jpg,None,EUGENIO JOSÉ ZULIANI
76,Sim,2021-10-06T16:49:06,160601,https://dadosabertos.camara.leg.br/api/v2/deputados/160601,Pr. Marco Feliciano,PL,https://dadosabertos.camara.leg.br/api/v2/partidos/37906,SP,56,https://www.camara.leg.br/internet/deputado/bandep/160601.jpg,dep.pr.marcofeliciano@camara.leg.br,MARCO ANTÔNIO FELICIANO
80,Sim,2021-10-06T16:49:00,204511,https://dadosabertos.camara.leg.br/api/v2/deputados/204511,David Soares,DEM,https://dadosabertos.camara.leg.br/api/v2/partidos/36769,SP,56,https://www.camara.leg.br/internet/deputado/bandep/204511.jpg,dep.davidsoares@camara.leg.br,DAVID BEZERRA RIBEIRO SOARES
100,Sim,2021-10-06T16:48:23,73604,https://dadosabertos.camara.leg.br/api/v2/deputados/73604,Rui Falcão,PT,https://dadosabertos.camara.leg.br/api/v2/partidos/36844,SP,56,https://www.camara.leg.br/internet/deputado/bandep/73604.jpg,dep.ruifalcao@camara.leg.br,RUI GOETHE DA COSTA FALCÃO
121,Sim,2021-10-06T16:47:41,204537,https://dadosabertos.camara.leg.br/api/v2/deputados/204537,Enrico Misasi,PV,https://dadosabertos.camara.leg.br/api/v2/partidos/36851,SP,56,https://www.camara.leg.br/internet/deputado/bandep/204537.jpg,None,ENRICO VAN BLARCUM DE GRAAFF MISASI
122,Sim,2021-10-06T16:47:40,204485,https://dadosabertos.camara.leg.br/api/v2/deputados/204485,Luiz Carlos Motta,PL,https://dadosabertos.camara.leg.br/api/v2/partidos/37906,SP,56,https://www.camara.leg.br/internet/deputado/bandep/204485.jpg,dep.luizcarlosmotta@camara.leg.br,LUIZ CARLOS MOTTA
147,Sim,2021-10-06T16:47:18,217036,https://dadosabertos.camara.leg.br/api/v2/deputados/217036,Henrique do Paraíso,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/partidos/37908,SP,56,https://www.camara.leg.br/internet/deputado/bandep/217036.jpg,None,HENRIQUE STEIN SCIASCIO
163,Sim,2021-10-06T16:47:06,73441,https://dadosabertos.camara.leg.br/api/v2/deputados/73441,Celso Russomanno,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/partidos/37908,SP,56,https://www.camara.leg.br/internet/deputado/bandep/73441.jpg,dep.celsorussomanno@camara.leg.br,CELSO UBIRAJARA RUSSOMANNO


# Ex. 1 - Raspagem dos dados por NOME DA PARTE

In [ ]:
driver = webdriver.Chrome()
driver.get('https://esaj.tjsp.jus.br/cpopg/open.do')
driver.maximize_window()

# --- LISTAS PARA ARMAZENAR DADOS ---
nome_deputados_lista = []
partidos_lista = []
num_processos_lista = []
classeprocesso_lista = []
assuntoprocesso_lista = []

def realizar_pesquisa(driver, nome_deputado):
    """Realiza a pesquisa por nome completo no site."""
    try:
        botao_tipo_pesquisa = driver.find_element(By.ID, "cbPesquisa")
        select_tipo_pesquisa = Select(botao_tipo_pesquisa)
        select_tipo_pesquisa.select_by_index(1)  # Seleciona para pesquisar por NOME

        campo_nome = driver.find_element(By.ID, "campo_NMPARTE")
        campo_nome.clear()
        campo_nome.send_keys(nome_deputado)
        
        driver.implicitly_wait(15)
        
        driver.find_element(By.ID, "pesquisarPorNomeCompleto").click() #Seleciona a caixa para pesquisar por nome completo
        
        driver.implicitly_wait(10)

        driver.find_element(By.ID, "botaoConsultarProcessos").click() #Clica no botão para pesquisar
        
    except NoSuchElementException:
        print(f"Erro ao realizar pesquisa para {nome_deputado}")
        return False  # Indica falha na pesquisa
    return True # Indica sucesso na pesquisa


def extrair_dados_da_tabela(driver, nome_deputado, partido_deputado):
    """Extrai dados da tabela de processos, se presente."""
    nome_lista_temp = []
    partidos_lista_temp = []
    num_processos_lista_temp = []
    classeprocesso_lista_temp = []
    assuntoprocesso_lista_temp = []

    try:
        lista_processo_elementos = driver.find_elements(By.CLASS_NAME, "linkProcesso")
        classe_processo_elementos = driver.find_elements(By.CLASS_NAME, "classeProcesso")
        assunto_processo_elementos = driver.find_elements(By.CLASS_NAME, "assuntoPrincipalProcesso")

        if not lista_processo_elementos: # Verifica se encontrou processos
            print(f"Nenhum processo encontrado na tabela para {nome_deputado}")
            return [], [], [], [], []

        for i in range(len(lista_processo_elementos)):
            nome_lista_temp.append(nome_deputado)
            partidos_lista_temp.append(partido_deputado)
            num_processos_lista_temp.append(lista_processo_elementos[i].text)
            classeprocesso_lista_temp.append(classe_processo_elementos[i].text)
            assuntoprocesso_lista_temp.append(assunto_processo_elementos[i].text)

    except NoSuchElementException:
        print(f"Erro ao extrair dados da tabela para {nome_deputado}")
        return [], [], [], [], [] # Retorna listas vazias em caso de erro

    return nome_lista_temp, partidos_lista_temp, num_processos_lista_temp, classeprocesso_lista_temp, assuntoprocesso_lista_temp


def extrair_detalhes_processo(driver, nome_deputado, partido_deputado):
    """Extrai detalhes do processo se estiver na página 'show.do?'."""
    try:
        nprocesso = driver.find_element(By.ID, "numeroProcesso").text
        processo_clase = driver.find_element(By.ID, "labelClasseProcesso").text
        processo_assunto = driver.find_element(By.ID, "assuntoProcesso").text

        nome_deputados_lista.append(nome_deputado)
        partidos_lista.append(partido_deputado)
        num_processos_lista.append(nprocesso)
        classeprocesso_lista.append(processo_clase)
        assuntoprocesso_lista.append(processo_assunto)

    except NoSuchElementException:
        print(f"Erro ao extrair detalhes do processo para {nome_deputado}")


def voltar_para_pesquisa(driver):
    """Limpa o campo de pesquisa para a próxima busca."""
    try:
        campo_nome = driver.find_element(By.ID, "campo_NMPARTE")
        campo_nome.clear()

        time.sleep(10)
        
    except NoSuchElementException:
        print("Erro ao limpar campo de pesquisa.")


# --- LOOP PRINCIPAL ---
for i, row in bd_deputados_sp.iterrows():  
    nome_deputado = row['NomeCivilDeputado']
    partido_deputado = row['deputado_.siglaPartido']

    print(f"Pesquisando por: {nome_deputado} ({partido_deputado})")

    if realizar_pesquisa(driver, nome_deputado): # Realiza pesquisa e verifica sucesso
        url = driver.current_url
        if "show.do?" in str(url):
            extrair_detalhes_processo(driver, nome_deputado, partido_deputado)
            try:
                driver.find_element(By.CLASS_NAME, 'unj-link-back').click() # Voltar para lista
            except NoSuchElementException:
                print("Erro ao clicar em 'Voltar' para lista de processos.")

        else: # Página de lista de processos (ou "nenhum processo encontrado")
            n, pa, np, cp, ap = extrair_dados_da_tabela(driver, nome_deputado, partido_deputado)
            nome_deputados_lista.extend(n)
            partidos_lista.extend(pa)
            num_processos_lista.extend(np)
            classeprocesso_lista.extend(cp)
            assuntoprocesso_lista.extend(ap)

        voltar_para_pesquisa(driver) # Limpa campo para próxima pesquisa
    else:
        print(f"Falha na pesquisa inicial para {nome_deputado}. Pulando para o próximo deputado.")

driver.quit()

Pesquisando por: NILTO IGNACIO TATTO (PT)
Erro ao realizar pesquisa para NILTO IGNACIO TATTO
Falha na pesquisa inicial para NILTO IGNACIO TATTO. Pulando para o próximo deputado.
Pesquisando por: FRANCISCO EVERARDO TIRIRICA OLIVEIRA SILVA (PL)
Erro ao realizar pesquisa para FRANCISCO EVERARDO TIRIRICA OLIVEIRA SILVA
Falha na pesquisa inicial para FRANCISCO EVERARDO TIRIRICA OLIVEIRA SILVA. Pulando para o próximo deputado.
Pesquisando por: EUGENIO JOSÉ ZULIANI (DEM)
Erro ao realizar pesquisa para EUGENIO JOSÉ ZULIANI
Falha na pesquisa inicial para EUGENIO JOSÉ ZULIANI. Pulando para o próximo deputado.
Pesquisando por: MARCO ANTÔNIO FELICIANO (PL)
Erro ao realizar pesquisa para MARCO ANTÔNIO FELICIANO
Falha na pesquisa inicial para MARCO ANTÔNIO FELICIANO. Pulando para o próximo deputado.
Pesquisando por: DAVID BEZERRA RIBEIRO SOARES (DEM)
Erro ao realizar pesquisa para DAVID BEZERRA RIBEIRO SOARES
Falha na pesquisa inicial para DAVID BEZERRA RIBEIRO SOARES. Pulando para o próximo deputad

### Criação do DataFrame

In [ ]:
data = {'Nome':nome_deputados_lista, 'Partido':partidos_lista, 'Processo':num_processos_lista, 'Classe':classeprocesso_lista, 'Assunto':assuntoprocesso_lista}

bd_deputados_sp = pd.DataFrame(data)

bd_deputados_sp

,Nome,Partido,Processo,Classe,Assunto


### Análise do DataFrame

In [ ]:
bd_deputados_sp['Assunto'].value_counts()

Series([], Name: count, dtype: int64)

In [ ]:
bd_deputados_sp['AssuntoProcesso'].value_counts().nlargest(50)

KeyError: 'AssuntoProcesso'

# Ex.2: Buscas por número do processo
Neste segundo exemplo utilizaremos dados obtidos via API do Conselho Nacional de Justiça (CNJ). 
O Datajud, API do CNJ, transfere dados via ElasticSearch. Segundo a própria documentação, "é a base nacional de metadados processuais do poder judiciário brasileiro. Sua finalidade é centralizar e padronizar as informações processuais de todo o país com o objetivo subsidiar o Sistema de Estatística do Poder Judiciário – SIESPJ conforme estabelecido pela Resolução CNJ n. 331/2020. A partir do Datajud é possível a geração de estatísticas e análises sobre a atividade judiciária em todo o território nacional."

Leia aqui os termos de uso da API: https://datajud-wiki.cnj.jus.br/api-publica/termo-uso

In [ ]:
# URL da API
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"

#Substituir <API Key> pela Chave Pública
headers = {
  'Authorization': 'APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==',
  'Content-Type': 'application/json'
}

In [ ]:
payload = json.dumps({
    "size":10000,
    "query": {
            "match":{"classe.codigo": 15167} ##Ações de Acordo de Não Persecução Civel
            },
    "sort": [{"dataAjuizamento": {"order":"asc"}}]
    }
)

In [ ]:
response = requests.request("POST", url, headers=headers, data=payload)
processo_json = response.json()

In [ ]:
processo_json['hits'].keys()

In [ ]:
processo_json['hits']['hits'][0]

In [ ]:
processos = []

for processo in processo_json['hits']['hits']:
  num_processo = processo['_source']['numeroProcesso']
  grau = processo['_source']['grau']
  assunto = processo['_source']['assuntos']
  num_classe_processo = processo['_source']['classe']['codigo']
  classe_processo = processo['_source']['classe']['nome']
  formato_processo = processo['_source']['formato']['nome']
  tribunal = processo['_source']['tribunal']
  data_ajuizamento = processo['_source']['dataAjuizamento']
  ultima_atualizacao = processo['_source']['dataHoraUltimaAtualizacao']
  orgao_julgador = processo['_source']['orgaoJulgador']['nome']
  sort = processo['sort'][0]
  try:
    movimentos = processo['_source']['movimentos']
  except:
    movimentos = []

  processos.append([num_processo, grau, num_classe_processo, classe_processo, formato_processo,
                      tribunal, data_ajuizamento, ultima_atualizacao, orgao_julgador, assunto, movimentos, sort])


bd = pd.DataFrame(processos, columns=['Numero_processo', 'Grau_processo', 'Numero_classeProcesso', 'Classe_processo', 'Formato_processo',
                                      'Tribunal', 'DataAjuizamento', 'UltimaAtualizacao', 'Orgao_julgador', 'Assunto_processo', 'Movimentos', 'Sort'])

bd.head(5)

In [ ]:
def converte_data(data_str):
    return pd.to_datetime(data_str).tz_convert('America/Sao_Paulo')


def gera_lista_assuntos(Assunto_processo):
  lista_assuntos = []
  for assunto in Assunto_processo:
    try:
      codigo = assunto.get('codigo')
      nome = assunto.get('nome')
      lista_assuntos.append([codigo, nome])
    except:
      lista_assuntos.append('')
  return lista_assuntos

def gera_lista_movimentos(Movimentos):
  lista_movimentos = []
  for movimento in Movimentos:
    codigo = movimento.get('codigo')
    nome = movimento.get('nome')
    data_hora = movimento.get('dataHora')

    if data_hora:
      data_hora = converte_data(data_hora)

    lista_movimentos.append([codigo, nome, data_hora])
  return lista_movimentos

In [ ]:
bd_processos['MesAjuizamento'] = pd.DatetimeIndex(bd_processos['DataAjuizamento']).month
bd_processos['AnoAjuizamento'] = pd.DatetimeIndex(bd_processos['DataAjuizamento']).year

In [ ]:
bd_processos = bd_processos[['Numero_processo', 'Grau_processo', 'Numero_classeProcesso',
         "Classe_processo", "Formato_processo", "Tribunal",
         "DataAjuizamento", "AnoAjuizamento", "MesAjuizamento",
         "UltimaAtualizacao", "Orgao_julgador", "Assunto_processo",
         "Movimentos", "Sort"]]

In [ ]:
for i, row in bd_processos.iterrows():
    nprocesso = (row['Numero_processo'][:13])
    nforo = (row['Numero_processo'][16:])
    
    print("Pesquisando por", nprocesso)

Pesquisando por 1000108682025
Pesquisando por 1000776342025
Pesquisando por 1000190912025
Pesquisando por 1000066252025
Pesquisando por 1000125632025
Pesquisando por 1000021152025
Pesquisando por 1000043812025
Pesquisando por 1522279732024
Pesquisando por 1003908702024
Pesquisando por 1006679262024
Pesquisando por 1501187422024
Pesquisando por 1010481492024
Pesquisando por 1058543882024
Pesquisando por 1036796822024
Pesquisando por 1002674842024
Pesquisando por 1002626282024
Pesquisando por 1092531892024
Pesquisando por 1502204222024
Pesquisando por 1514463512024
Pesquisando por 1011468612024
Pesquisando por 1022234062024
Pesquisando por 1017564062024
Pesquisando por 1013647182024
Pesquisando por 1089085782024
Pesquisando por 1001552472024
Pesquisando por 1088842372024
Pesquisando por 1004521102024
Pesquisando por 1087219352024
Pesquisando por 1002473842024
Pesquisando por 1028931402024
Pesquisando por 1015642452024
Pesquisando por 1006136892024
Pesquisando por 1508885172024
Pesquisand

In [ ]:
entidades = [] 
driver = webdriver.Chrome()
driver.get('https://esaj.tjsp.jus.br/cpopg/open.do')
driver.maximize_window()

def pesquisa(driver, nprocesso, nforo):
    try:
        botao_tipo_pesquisa = driver.find_element(By.ID, "cbPesquisa")
        select_tipo_pesquisa = Select(botao_tipo_pesquisa)
        select_tipo_pesquisa.select_by_index(0)  # Seleciona para pesquisar por NÚMERO DO PROCESSO

        campo_nprocesso = driver.find_element(By.ID, 'numeroDigitoAnoUnificado')
        campo_foronumero = driver.find_element(By.ID, 'foroNumeroUnificado')

        campo_nprocesso.clear()
        campo_foronumero.clear()

        campo_nprocesso.send_keys(nprocesso)
        campo_foronumero.send_keys(nforo)

        time.sleep(5)

        search_button = driver.find_element(By.ID, "botaoConsultarProcessos") #Clica no botão para pesquisar
        search_button.click()

    except NoSuchElementException:
        print(f"Erro ao realizar pesquisa para {nprocesso, nforo}")
        return False  # Indica falha na pesquisa
    
    return True

def extrai_infos(driver, entidades):
    time.sleep(5)
    partes_processo = driver.find_elements(By.CLASS_NAME, 'nomeParteEAdvogado')   
    for partes in partes_processo:
        print(partes.text)
        entidades.append(partes.text)

def voltar_pesquisa(driver):
    back_button = driver.find_element(By.ID, 'setaVoltar')
    back_button.click()

#Raspagem
count = 1 
for i, row in bd_processos.iterrows():
    nprocesso = (row['Numero_processo'][:13])
    nforo = (row['Numero_processo'][16:])
    
    print(count, "Pesquisando por", nprocesso+nforo)
    count += 1 

    pesquisa(driver, nprocesso, nforo)
    extrai_infos(driver, entidades)
    voltar_pesquisa(driver)

Pesquisando por 10001086820250185
Ministério Público do Estado de São Paulo
Adauto Severo Pinto
MUNICIPIO DE POPULINA
Pesquisando por 10007763420250510
PREFEITURA MUNICIPAL DE RIO CLARO
Advogada:  Rosemari Ap Castello da Silva  
Gustavo Brito da Cunha
Pesquisando por 10001909120250123
PREFEITURA MUNICIPAL DE CAPÃO BONITO
Advogada:  Luana Maria Rodrigues  
Alexandrina Maria Aparecida Citadini
Pesquisando por 10000662520250280
Ministério Público do Estado de São Paulo
Ferpel Engenharia e Construções Ltda. - Epp





Pesquisando por 10001256320250619
Ministério Público do Estado de São Paulo
Vanderlei José Marsico (Prefeito do Município de Taquaritinga)
Prefeitura Municipal de Taquaritinga







Pesquisando por 10000211520250185
Ministério Público do Estado de São Paulo
Marcos Antonio Saes Lopes
Pesquisando por 10000438120250344
Joseli Damasceno Abib
Advogada:  Ercilia Aparecida Pigozzi Garcia  
Pesquisando por 15222797320240224
Ministério Público do Estado de São Paulo
Jesus Roque de Fr

ElementClickInterceptedException: Message: element click intercepted: Element <i id="setaVoltar" class="icon-back"></i> is not clickable at point (255, 102). Other element would receive the click: <div class="blockUI blockOverlay" style="z-index: 1000; border: none; margin: 0px; padding: 0px; width: 100%; height: 100%; top: 0px; left: 0px; background-color: rgb(0, 0, 0); opacity: 0.4; cursor: default; position: fixed;"></div>
  (Session info: chrome=133.0.6943.127)
Stacktrace:
	GetHandleVerifier [0x00007FF636636EE5+28773]
	(No symbol) [0x00007FF6365A25D0]
	(No symbol) [0x00007FF636438FAA]
	(No symbol) [0x00007FF636497109]
	(No symbol) [0x00007FF636494AD2]
	(No symbol) [0x00007FF636491B81]
	(No symbol) [0x00007FF636490A91]
	(No symbol) [0x00007FF636482284]
	(No symbol) [0x00007FF6364B724A]
	(No symbol) [0x00007FF636481B36]
	(No symbol) [0x00007FF6364B7460]
	(No symbol) [0x00007FF6364DF6F3]
	(No symbol) [0x00007FF6364B7023]
	(No symbol) [0x00007FF63647FF5E]
	(No symbol) [0x00007FF6364811

In [ ]:
entidades

['Ministério Público do Estado de São Paulo',
 'Adauto Severo Pinto',
 'MUNICIPIO DE POPULINA',
 'PREFEITURA MUNICIPAL DE RIO CLARO\nAdvogada:  Rosemari Ap Castello da Silva  ',
 'Gustavo Brito da Cunha',
 'PREFEITURA MUNICIPAL DE CAPÃO BONITO\nAdvogada:  Luana Maria Rodrigues  ',
 'Alexandrina Maria Aparecida Citadini',
 'Ministério Público do Estado de São Paulo',
 'Ferpel Engenharia e Construções Ltda. - Epp',
 '',
 '',
 '',
 '',
 '',
 'Ministério Público do Estado de São Paulo',
 'Vanderlei José Marsico (Prefeito do Município de Taquaritinga)',
 'Prefeitura Municipal de Taquaritinga',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Ministério Público do Estado de São Paulo',
 'Marcos Antonio Saes Lopes',
 'Joseli Damasceno Abib\nAdvogada:  Ercilia Aparecida Pigozzi Garcia  ',
 'Ministério Público do Estado de São Paulo',
 'Jesus Roque de Freitas',
 'Município de Guarulhos',
 '',
 '',
 '',
 '',
 'Ministério Público do Estado de São Paulo',
 'Ovídio Alexandre Azzini',
 'PREFEITURA MUNICIPAL DE 